# Dream KG - Homelessness


## Setup

In [2]:
!pip install neo4j pgeocode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 14.5 MB/s eta 0:00:00


## General

In [3]:
from functools import reduce
from neo4j import GraphDatabase
import pandas as pd
import json
import pgeocode

In [4]:
NEO4J_URI='neo4j+s://c16e1829.databases.neo4j.io'
NEO4J_USERNAME='neo4j'
NEO4J_PASSWORD='EsDnKL2ZIC5O4GZlu7qJqvww3ZI_igU0PHa6calBW-k'
AURA_INSTANCEID='c16e1829'
AURA_INSTANCENAME='Instance01'
default_end_date = 4102444799000

In [5]:
driver = GraphDatabase.driver(uri = NEO4J_URI, auth = (NEO4J_USERNAME, NEO4J_PASSWORD), database = 'neo4j')

In [8]:
def str_rem_spec_char(val: str):
  return val.replace('(', '').replace(')', '').replace('-', '').replace(' ', '') if val and pd.notna(val)  else ''

def str_strip(val):
  return val.strip() if val and pd.notna(val)  else ''

def get_node_unique_identifiers(session, node, node_type):
  query = '''
  MATCH (node:DREAM_KG:RESOURCE{nodeName: $node, nodeType: $node_type}) RETURN COLLECT({identifier: node.identifier, abbreviation: node.abbreviation}) AS dataList
  '''
  dataList = list(session.run(query, node=node, node_type=node_type))[0]['dataList']
  if len(dataList) == 1:
    return True, dataList[0]
  elif len(dataList) > 1:
    return False, dataList
  else:
    return False, None

def get_meta(node_name: str, node_type: str, cache_dict: dict):
  if node_name in cache_dict:
    res = True, cache_dict[node_name]['meta']
  else:
    res = get_node_unique_identifiers(session, node_name.strip(), node_type)
    if res[0]:
      cache_dict[node_name] = dict()
      cache_dict[node_name]['meta'] = res
    return res

In [9]:
create_thing_node = '''
MERGE (t:THING{
  nodeName: "THING"
}) SET t += {
  createdDate: datetime(),
  createdBy: "system",
  startTime: timestamp(),
  endTime: 4102444799000,
  nodeDesc: "RootNode",
  elementType: "class"
}
RETURN COUNT(*)
'''
create_category_node = '''
MERGE (t:THING{nodeName: "THING"})
MERGE (n:METADATA:RESOURCE{
  nodeName: "DREAM_KG",
  identifier: "dreamkg",
  abbreviation: "dreamkg",
  elementType: "class",
  nodeCategoryName: "DREAM_KG",
  metadataType: "nodeCategory"
}) SET n += {
  startTime: timestamp(),
  endTime: 4102444799000,
  nodeCategory: 1,
  createdDate: datetime(),
  createdBy: "system",
  nodeCategoryDescription: "DREAM-KG can be used to provide precision social services and resources available for people
experiencing homelessness (PEH), including emergency shelters, transitional housing, permanent
supportive housing, food assistance programs, healthcare services, employment and job training, legal
assistance, and substance abuse and mental health services."
}
MERGE (t)-[r:hasSubClass]->(n) SET r += {
  startTime: timestamp(),
  endTime: 4102444799000,
  createdBy: "system"
}
RETURN COUNT(*)
'''
def node_upsert(**kwargs):
  if not kwargs.get('subClass', False):
    label = ':DREAM_KG:RESOURCE'
    subclass = ':METADATA:RESOURCE{nodeTypeName: data.nodeType, metadataType: "nodeType"}'
    rel = 'hasInstance'
  else:
    label = ':METADATA:RESOURCE'
    subclass = ':METADATA:RESOURCE{nodeCategoryName: "DREAM_KG", metadataType: "nodeCategory"}'
    rel = 'hasSubClass'

  query = '''
  WITH $data AS data, {
    createdDate: timestamp(),
    startTime: timestamp(),
    endTime: 4102444799000,
    status: "Confirmed"
  } AS enrichData
  MATCH (nodeClass%s)'''
  if 'parent' in kwargs:
    query += ('''
    MATCH (parent:DREAM_KG:RESOURCE{identifier: "%s", abbreviation: "%s"})
    ''' % (kwargs['parent']['identifier'], kwargs['parent']['abbreviation']))
  query += '''
  MERGE (node%s{identifier: nodeClass.identifier + '.' + TOLOWER(apoc.text.regreplace(data.nodeName, '[^a-zA-Z0-9]', '')),'''
  query += ('''
                abbreviation: %s + TOLOWER(apoc.text.regreplace(data.nodeName, '[^a-zA-Z0-9]', ''))})
  ''' % ('COALESCE(parent.abbreviation, nodeClass.abbreviation)' + '+ \'.\'' if 'parent' in kwargs else '\'dreamkg.\''))
  query += '''
  ON CREATE
    SET node.operation = 'new_node'
  ON MATCH
    SET node.operation = 'update_node'
  SET node += enrichData
  SET node += data
  MERGE (node)-[:_properties{audit: true}]->(audit:AUDIT)
  CREATE (audit)-[:_time_properties]->(spoke:SPOKE) SET spoke += PROPERTIES(node)
  MERGE (node)<-[r:%s]-(nodeClass) ON CREATE SET r += enrichData
  '''
  if 'parent' in kwargs:
    query += '''
    MERGE (parent)-[pr:hasChild]->(node) ON CREATE SET pr += enrichData
    '''
  query += '''
  RETURN DISTINCT node
  '''
  query = query % (subclass, label, rel)
  return query


def business_rel_upsert(parent, rel, child):
  return '''WITH $data AS data, {
    createdDate: timestamp(),
    startTime: timestamp(),
    endTime: 4102444799000,
    status: "Confirmed"
  } AS enrichData
  MATCH (parent:DREAM_KG:RESOURCE{identifier: "%s", abbreviation: "%s"})
  MATCH (child:DREAM_KG:RESOURCE{identifier: "%s", abbreviation: "%s"})
  MERGE (parent)-[rel:%s]->(child)
  SET rel += enrichData SET rel += data
  RETURN COUNT(*)
  ''' % (parent['identifier'], parent['abbreviation'], child['identifier'], child['abbreviation'], rel)

### Root Node Creation

In [7]:
def transaction_call(tx, query):
    result = tx.run(query)
with driver.session() as session:
  session.execute_write(transaction_call, create_thing_node)
  session.execute_write(transaction_call, create_category_node)

### Node Types Creation

In [8]:
# Node Type Creation
with driver.session() as session:
  session.run(node_upsert(subClass=True),
                          data={"nodeName": "Amenity",
                                "isRootNodeType": True,
                                "elementType": "class",
                                "nodeTypeName": "Amenity",
                                "nodeCategory": "DREAM_KG",
                                "metadataType": "nodeType",
                                "parentNodeType": "Amenity",
                                "childNodeType": "Amenity"})
  session.run(node_upsert(subClass=True),
                          data={"nodeName": "Organization",
                                "isRootNodeType": True,
                                "elementType": "class",
                                "nodeTypeName": "Organization",
                                "nodeCategory": "DREAM_KG",
                                "metadataType": "nodeType",
                                "parentNodeType": "",
                                "childNodeType": "ProgramOrFacility"})
  session.run(node_upsert(subClass=True),
                          data={"nodeName": "ProgramOrFacility",
                                "elementType": "class",
                                "nodeTypeName": "ProgramOrFacility",
                                "nodeCategory": "DREAM_KG",
                                "metadataType": "nodeType",
                                "parentNodeType": "Organization",
                                "childNodeType": "Service"})
  session.run(node_upsert(subClass=True),
                          data={"nodeName": "Service",
                                "elementType": "class",
                                "nodeTypeName": "Service",
                                "nodeCategory": "DREAM_KG",
                                "metadataType": "nodeType",
                                "parentNodeType": "Service",
                                "childNodeType": ""})
  session.run(node_upsert(subClass=True),
                          data={"nodeName": "Neighborhood",
                                "isRootNodeType": True,
                                "elementType": "class",
                                "nodeTypeName": "Neighborhood",
                                "nodeCategory": "DREAM_KG",
                                "metadataType": "nodeType",
                                "parentNodeType": "Neighborhood",
                                "childNodeType": "Neighborhood"})
  session.run(node_upsert(subClass=True),
                          data={"nodeName": "Location",
                                "isRootNodeType": True,
                                "elementType": "class",
                                "nodeTypeName": "Location",
                                "nodeCategory": "DREAM_KG",
                                "metadataType": "nodeType",
                                "parentNodeType": "Location",
                                "childNodeType": "Location"})

In [9]:
ORG_NODE_TYPE = 'Organization'
CAT_NODE_TYPE = 'Amenities'
NH_NODE_TYPE = 'Neighborhood'
LOC_NODE_TYPE = 'Location'

root_cache_dict = dict()
root_cache_dict[ORG_NODE_TYPE] = dict()
root_cache_dict[CAT_NODE_TYPE] = dict()
root_cache_dict[NH_NODE_TYPE] = dict()
root_cache_dict[LOC_NODE_TYPE] = dict()

org_dict = root_cache_dict[ORG_NODE_TYPE]
cat_dict = root_cache_dict[CAT_NODE_TYPE]
nh_dict = root_cache_dict[NH_NODE_TYPE]
loc_dict = root_cache_dict[LOC_NODE_TYPE]

## Sheet 1 - 5a05b9ec-2fbf-43f2-bfff-1de2555ff7d4

In [10]:
SRC_SHEET = 'dataset/transformed/5a05b9ec-2fbf-43f2-bfff-1de2555ff7d4.xlsx'
df = pd.read_excel(SRC_SHEET, header=0, skiprows=[0])

In [11]:
with driver.session() as session:
  for org in df['organization'].unique():
    org_node = list(session.run(node_upsert(),
                          data={"nodeName": str_strip(org),
                            "elementType": "instance",
                            "nodeType": "Organization",
                            "nodeCategory": "DREAM_KG",
                            "srcSheet": SRC_SHEET
                          }))[0]['node']
    root_cache_dict[ORG_NODE_TYPE][org] = {
        'identifier': org_node.get('identifier'),
        'abbreviation': org_node.get('abbreviation')
    }
  for category in set(reduce(lambda x,y: x+y, [i.split('|') for i in df['category'].unique()])):
    cat_node = list(session.run(node_upsert(),
                          data={"nodeName": str_strip(category),
                            "elementType": "instance",
                            "nodeType": "Amenity",
                            "nodeCategory": "DREAM_KG",
                            "srcSheet": SRC_SHEET
                          }))[0]['node']
    root_cache_dict[CAT_NODE_TYPE][category] = {
        'identifier': cat_node.get('identifier'),
        'abbreviation': cat_node.get('abbreviation')
    }
  for neighborhood in df['neighborhood'].unique():
    nh_node = list(session.run(node_upsert(),
                          data={"nodeName": str_strip(neighborhood),
                            "elementType": "instance",
                            "nodeType": "Neighborhood",
                            "nodeCategory": "DREAM_KG",
                            "srcSheet": SRC_SHEET
                          }))[0]['node']
    root_cache_dict[NH_NODE_TYPE][neighborhood] = {
        'identifier': nh_node.get('identifier'),
        'abbreviation': nh_node.get('abbreviation')
    }

In [12]:
with driver.session() as session:
  for org, fac in df[['organization', 'program_or_facility']].drop_duplicates().values:
    fac_node = list(session.run(node_upsert(parent=org_dict[str_strip(org)]),
                          data={"nodeName": str_strip(fac),
                            "elementType": "instance",
                            "nodeType": "ProgramOrFacility",
                            "nodeCategory": "DREAM_KG",
                            "srcSheet": SRC_SHEET
                          }))[0]['node']
    fac_meta = {
        'identifier': fac_node.get('identifier'),
        'abbreviation': fac_node.get('abbreviation')
    }
    org_dict[org][fac] = fac_meta

In [13]:
with driver.session() as session:
  for index, row in df.iterrows():
    prg_meta = org_dict[str_strip(row['organization'])][str_strip(row['program_or_facility'])]

    serv_node = list(session.run(node_upsert(parent=prg_meta),
                      data={"nodeName": str_strip(row['service_name']),
                        "elementType": "instance",
                        "nodeType": "Service",
                        "nodeCategory": "DREAM_KG",
                        "address": str_strip(row["address"]),
                        "latitude": row["latitude"],
                        "longitude": row["longitude"],
                        "recommendedFor": str_strip(row["recommended_for"]),
                        "requirements": str_strip(row["requirements"]),
                        "phone": str_strip(str_rem_spec_char(row["phone"])) if row['phone'] else '',
                        "nodeDesc": str_strip(row['narrative']),
                        "schedule": str_strip(row['schedule']),
                        "holidayException": str_strip(row['holiday_exception']),
                        "srcSheet": SRC_SHEET
                      }))[0]['node']
    serv_meta = {
        'identifier': serv_node.get('identifier'),
        'abbreviation': serv_node.get('abbreviation')
    }
    for cat in row['category'].split('|'):
      cat_meta = cat_dict[cat]
      session.run(business_rel_upsert(parent=serv_meta, child=cat_meta, rel='hasAmenity'), data={})

    nh_meta = nh_dict[row['neighborhood']]
    session.run(business_rel_upsert(parent=serv_meta, child=nh_meta, rel='hasNeighborhood'), data={})

## Sheet 2 - Food_Resources_in_California.csv

In [14]:
SRC_SHEET = 'dataset/Food_Resources_in_California.csv'
food_data_ca_df = pd.read_csv(SRC_SHEET)

In [15]:
food_data_ca_df['web_link_extract'] = food_data_ca_df['Web Link'].str.extract(r'https?://w?w?w?[.]?([^.]+)')

In [16]:
food_data_ca_df.loc[food_data_ca_df['web_link_extract'] == 'facebook', 'web_link_extract'] = food_data_ca_df.loc[food_data_ca_df['web_link_extract'] == 'facebook', 'Web Link'].str.extract(r'(?i)facebook.com/?(?:pg|pages)?/?(?:category)?/?(?:community)?/?([^/]*)?')[0]
food_data_ca_df['web_link_extract'] = food_data_ca_df['web_link_extract'].fillna(food_data_ca_df['Name'])

In [17]:
food_data_ca_df.loc[[19, 144, 198]]

,Name,Street Address,City,State,Zip Code,County,Phone,Description,Resource Type,Web Link,Notes,Latitude,Longitude,web_link_extract
19,Catholic Ladies Relief Society,1386 Longfellow Avenue,Chico,CA,95926,Butte,(530) 895-8331,"Monday - Friday, 9am-12pm. Emergency food assi...",Food Distribution Program,https://www.facebook.com/Catholic-Ladies-Relie...,NaN,39.751954,-121.824637,Catholic-Ladies-Relief-Society-3-1681359945507214
144,Rural Human Services (RHS),286 M Street Suite A,Crescent City,CA,95531,Del Norte,707-464-7441,Offers emergency food boxes and basic hygiene ...,Food Pantry,http://www.facebook.com/rhsfoodbank/,NaN,41.754594,-124.192643,rhsfoodbank
198,Our Savior’s Lutheran\nChurch (Food Resources),425 Shaw Avenue,Ferndale,CA,95536,Humboldt,707-786-9619,Free clothing closet open on Wednesdays from 1...,Food Pantry,NaN,NaN,40.578274,-124.264446,Our Savior’s Lutheran\nChurch (Food Resources)


In [18]:
food_data_ca_df.loc[(food_data_ca_df['web_link_extract'].isna()) | (food_data_ca_df['web_link_extract'] == 'facebook')]

,Name,Street Address,City,State,Zip Code,County,Phone,Description,Resource Type,Web Link,Notes,Latitude,Longitude,web_link_extract


In [19]:
# Organization
with driver.session() as session:
  for org in food_data_ca_df['web_link_extract'].unique():
    org_node = list(session.run(node_upsert(),
                          data={"nodeName": str_strip(org),
                            "elementType": "instance",
                            "nodeType": "Organization",
                            "nodeCategory": "DREAM_KG",
                            "srcSheet": SRC_SHEET
                          }))[0]['node']
    root_cache_dict[ORG_NODE_TYPE][org] = {
        'identifier': org_node.get('identifier'),
        'abbreviation': org_node.get('abbreviation')
    }

# ProgramOrFacility
with driver.session() as session:
  for org, fac in food_data_ca_df[['web_link_extract', 'Name']].drop_duplicates().values:
    fac_node = list(session.run(node_upsert(parent=org_dict[str_strip(org)]),
                          data={"nodeName": str_strip(fac),
                            "elementType": "instance",
                            "nodeType": "ProgramOrFacility",
                            "nodeCategory": "DREAM_KG",
                            "srcSheet": SRC_SHEET
                          }))[0]['node']
    fac_meta = {
        'identifier': fac_node.get('identifier'),
        'abbreviation': fac_node.get('abbreviation')
    }
    org_dict[org][fac] = fac_meta

In [20]:
food_data_ca_df['Phone'] = food_data_ca_df['Phone'].apply(lambda x: str_rem_spec_char(x))
# Aggregate to merge same service records - 13 duplicate records found
food_data_ca_df_grouped = food_data_ca_df.groupby(['web_link_extract', 'Name', 'Resource Type', 'Street Address', 'City', 'State', 'Zip Code', 'County']).agg({
    'Phone': set,
    'Web Link': set,
    'Description': set,
    'Notes': set,
    'Latitude': set,
    'Longitude': set,
})
food_data_ca_df_grouped = food_data_ca_df_grouped.reset_index()
food_data_ca_df_grouped.fillna('', inplace=True)
food_data_ca_df_grouped['address'] = food_data_ca_df_grouped['Street Address'] + ', ' + food_data_ca_df_grouped['City'] + ', ' + food_data_ca_df_grouped['State'] + ' ' + food_data_ca_df_grouped['Zip Code'].astype(str)

In [21]:
food_data_ca_df_grouped['address'] = food_data_ca_df_grouped['Street Address'] + ', ' + food_data_ca_df_grouped['City'] + ', ' + food_data_ca_df_grouped['State'] + ' ' + food_data_ca_df_grouped['Zip Code'].astype(str)
with driver.session() as session:
  for category in food_data_ca_df_grouped['Resource Type'].unique():
    cat_node = list(session.run(node_upsert(),
                          data={"nodeName": str_strip(category),
                            "elementType": "instance",
                            "nodeType": "Amenity",
                            "nodeCategory": "DREAM_KG",
                            "srcSheet": SRC_SHEET
                          }))[0]['node']
    root_cache_dict[CAT_NODE_TYPE][category] = {
        'identifier': cat_node.get('identifier'),
        'abbreviation': cat_node.get('abbreviation')
    }

In [22]:
with driver.session() as session:
  for index, row in food_data_ca_df_grouped.iterrows():
    prg_meta = org_dict[str_strip(row['web_link_extract'])][str_strip(row['Name'])]

    serv_node = list(session.run(node_upsert(parent=prg_meta),
                      data={"nodeName": str_strip(row['Resource Type']) + ' - ' + str_strip(row['Street Address']),
                        "elementType": "instance",
                        "nodeType": "Service",
                        "nodeCategory": "DREAM_KG",
                        "nodeDesc": str_strip('; '.join([str(val) if pd.notna(val) else '' for val in row['Description']])),
                        "url": str_strip(';'.join([str(val) if pd.notna(val) else '' for val in row["Web Link"]])),
                        "notes": str_strip(';'.join([str(val) if pd.notna(val) else '' for val in row['Notes']])),
                        "category": str_strip(row['Resource Type']),
                        "streetAddress": str_strip(row["Street Address"]),
                        "city": str_strip(row["City"]),
                        "state": str_strip(row["State"]),
                        "zipCode": row["Zip Code"],
                        "county": str_strip(row["County"]),
                        "phone": list(row['Phone']),
                        "latitude": list(row["Latitude"]),
                        "longitude": list(row["Longitude"]),
                        "srcSheet": SRC_SHEET
                      }))[0]['node']
    serv_meta = {
        'identifier': serv_node.get('identifier'),
        'abbreviation': serv_node.get('abbreviation')
    }
    session.run(business_rel_upsert(parent=serv_meta, child=cat_dict[str_strip(row['Resource Type'])], rel='hasAmenity'), data={})

## Sheet 3 - FOOD BANKS.csv


In [23]:
SRC_SHEET = 'dataset/FOOD BANKS.csv'
food_banks_df = pd.read_csv(SRC_SHEET)
food_banks_df.fillna('', inplace=True)

<ipython-input-23-2fe2373edb00>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  food_banks_df.fillna('', inplace=True)


In [24]:
# Category
with driver.session() as session:
  for category in set(reduce(lambda x,y: x+y, [i.split(';') for i in food_banks_df['KEYWORDS'].unique()])):
    cat_node = list(session.run(node_upsert(),
                          data={"nodeName": str_strip(category),
                            "elementType": "instance",
                            "nodeType": "Amenity",
                            "nodeCategory": "DREAM_KG",
                            "srcSheet": SRC_SHEET
                          }))[0]['node']
    root_cache_dict[CAT_NODE_TYPE][category] = {
        'identifier': cat_node.get('identifier'),
        'abbreviation': cat_node.get('abbreviation')
    }

In [25]:
with driver.session() as session:
  for org in food_banks_df['CUST_ORG'].unique():
    org_node = list(session.run(node_upsert(),
                          data={"nodeName": str_strip(org),
                            "elementType": "instance",
                            "nodeType": "Organization",
                            "nodeCategory": "DREAM_KG",
                            "srcSheet": SRC_SHEET
                          }))[0]['node']
    root_cache_dict[ORG_NODE_TYPE][org] = {
        'identifier': org_node.get('identifier'),
        'abbreviation': org_node.get('abbreviation')
    }

In [26]:
# ProgramOrFacility
org_dict = root_cache_dict[ORG_NODE_TYPE]
with driver.session() as session:
  for org, fac in food_banks_df[['CUST_ORG', 'FCLTY_NM']].drop_duplicates().values:
    fac_node = list(session.run(node_upsert(parent=org_dict[org]),
                          data={"nodeName": str_strip(fac),
                            "elementType": "instance",
                            "nodeType": "ProgramOrFacility",
                            "nodeCategory": "DREAM_KG",
                            "srcSheet": SRC_SHEET
                          }))[0]['node']
    fac_meta = {
        'identifier': fac_node.get('identifier'),
        'abbreviation': fac_node.get('abbreviation')
    }
    org_dict[org][fac] = fac_meta

In [27]:
with driver.session() as session:
  for index, row in food_banks_df.iterrows():
    prg_meta = org_dict[row['CUST_ORG']][str_strip(row['FCLTY_NM'])]

    serv_node = list(session.run(node_upsert(parent=prg_meta),
                      data={"nodeName": str_strip(row['FCLTY_NM']),
                        "elementType": "instance",
                        "nodeType": "Service",
                        "nodeCategory": "DREAM_KG",
                        "address": str_strip(row["ADDRESS"]),
                        "streetAddress": str_strip(row["ST_ADDRESS"]),
                        "city": str_strip(row["LOCALITY"]),
                        "mailAddress": str_strip(row["MAIL_ADD"]),
                        "zipCode": str_strip(row["POSTAL_CD"]),
                        "nodeDesc": str_strip(row['BUS_CAT_DS']),
                        "url": str_strip(row['WEBSITE']),
                        "imageUrl": str_strip(row['IMAGE_URL']),
                        "phone": row['CONT_PHONE'],
                        "fax": row['CONT_FAX'],
                        "latitude": row["LATITUDE"],
                        "longitude": row["LONGITUDE"],
                        "xCoordinate": row["X"],
                        "yCoordinate": row["Y"],
                        "srcUpdatedDate": row['DT_UPDATE'],
                        "srcId": row['SRCDATA_ID'],
                        "srcSheet": SRC_SHEET
                      }))[0]['node']
    serv_meta = {
        'identifier': serv_node.get('identifier'),
        'abbreviation': serv_node.get('abbreviation')
    }

    for cat in row['KEYWORDS'].split(';'):
      cat_meta = cat_dict[str_strip(cat)]
      session.run(business_rel_upsert(parent=serv_meta, child=cat_meta, rel='hasAmenity'), data={})

## Sheet 4 - Directory_Of_Homeless_Drop-_In_Centers.csv

In [28]:
SRC_SHEET = 'dataset/Directory_Of_Homeless_Drop-_In_Centers.csv'
dir_homeless_center_df = pd.read_csv(SRC_SHEET)
dir_homeless_center_df.fillna('', inplace=True)

In [29]:
org_dict = root_cache_dict[ORG_NODE_TYPE]
with driver.session() as session:
  org_node = list(session.run(node_upsert(),
                        data={"nodeName": "Other",
                          "elementType": "instance",
                          "nodeType": "Organization",
                          "nodeCategory": "DREAM_KG",
                          "srcSheet": SRC_SHEET
                        }))[0]['node']
  org_dict['Other'] = {
      'identifier': org_node.get('identifier'),
      'abbreviation': org_node.get('abbreviation')
  }

  fac_node = list(session.run(node_upsert(parent=org_dict['Other']),
                        data={"nodeName": 'Other',
                          "elementType": "instance",
                          "nodeType": "ProgramOrFacility",
                          "nodeCategory": "DREAM_KG",
                          "srcSheet": SRC_SHEET
                        }))[0]['node']
  fac_meta = {
      'identifier': fac_node.get('identifier'),
      'abbreviation': fac_node.get('abbreviation')
  }
  org_dict['Other']['Other'] = fac_meta

  cat_node = list(session.run(node_upsert(),
                        data={"nodeName": "Drop-In Center",
                          "elementType": "instance",
                          "nodeType": "Amenity",
                          "nodeCategory": "DREAM_KG",
                          "srcSheet": SRC_SHEET
                        }))[0]['node']
  root_cache_dict[CAT_NODE_TYPE]['Drop-In Center'] = {
      'identifier': cat_node.get('identifier'),
      'abbreviation': cat_node.get('abbreviation')
  }

In [30]:
org_dict = root_cache_dict[ORG_NODE_TYPE]
with driver.session() as session:
  for index, row in dir_homeless_center_df.iterrows():
    prg_meta = org_dict['Other']['Other']

    serv_node = list(session.run(node_upsert(parent=prg_meta),
                      data={"nodeName": str_strip(row['Center Name']) + ' - ' + str_strip(row['Borough']),
                        "elementType": "instance",
                        "nodeType": "Service",
                        "nodeCategory": "DREAM_KG",
                        "address": str_strip(row["Address"]),
                        "city": str_strip(row["Borough"]),
                        "neighborhood": str_strip(row["NTA"]),
                        "zipCode": row["Postcode"],
                        "nodeDesc": str_strip(row['Comments']),
                        "communityBoard": row['Community Board'],
                        "councilDistrict": row['Council District'],
                        "censusTract": str_strip(row['Census Tract'] if pd.isna(row['Census Tract']) else ''),
                        "latitude": row["Latitude"],
                        "longitude": row["Longitude"],
                        "bin": row["BIN"],
                        "bbl": row["BBL"],
                        "srcSheet": SRC_SHEET
                      }))[0]['node']
    serv_meta = {
        'identifier': serv_node.get('identifier'),
        'abbreviation': serv_node.get('abbreviation')
    }
    session.run(business_rel_upsert(parent=serv_meta, child=root_cache_dict[CAT_NODE_TYPE]['Drop-In Center'], rel='hasAmenity'), data={})

## Data Enrichment


MATCH (n:DREAM_KG{nodeType: "Service", srcSheet: 'dataset/Directory_Of_Homeless_Drop-_In_Centers.csv'}) WHERE n.streetAddress IS NULL WITH n, [i in apoc.text.split(n.address, '[;,]') | trim(i)] AS text WITH n, text[0] AS streetAddress, text[1] AS city, apoc.text.split(text[2], ' ') AS addr2 WITH n, streetAddress, city, TOINTEGER(addr2[1]) AS zipCode SET n += {streetAddress: streetAddress, city: city, state: 'New York', zipCode: zipCode} RETURN n.address, n.streetAddress, n.city, n.state, n.zipCode


MATCH (n:DREAM_KG{nodeType: "Service", srcSheet: "dataset/transformed/5a05b9ec-2fbf-43f2-bfff-1de2555ff7d4.xlsx"}) WHERE n.streetAddress IS NULL
WITH n, [i in apoc.text.split(n.address, ', ') | trim(i)] AS text WHERE text[1] = 'PA'
WITH n, text SET n += {streetAddress: text[0], state: 'Pennsylvania' , zipCode: TOINTEGER(text[2])}
RETURN n.address, n.streetAddress, n.city, n.state, n.zipCode ORDER BY n.zipCode DESC


MATCH (n:DREAM_KG{nodeType: "Service", srcSheet: "dataset/transformed/5a05b9ec-2fbf-43f2-bfff-1de2555ff7d4.xlsx"}) WHERE n.streetAddress IS NULL
WITH n, [i in apoc.text.split(n.address, ', ') | trim(i)] AS text WHERE text[1] <> 'PA'
WITH n, text SET n += {streetAddress: text[0], state: 'Pennsylvania' , zipCode: TOINTEGER(text[-1])}
RETURN n.address, text, n.streetAddress, n.city, n.zipCode ORDER BY n.zipCode DESC

MATCH (n:DREAM_KG{nodeType: "Service", city: 'NY 10012'}) SET n.zipCode = 10012 REMOVE n.city RETURN n

MATCH (n:DREAM_KG{nodeType: 'Location'}) WITH KEYS(n) AS keys UNWIND keys AS key
RETURN COLLECT(DISTINCT key)


**Enrichment Queries**

MATCH (n:DREAM_KG{nodeType: 'Service'})
MATCH (loc:DREAM_KG{nodeType: 'Location'}) WHERE TOSTRING(n.zipCode) CONTAINS loc.nodeName
MERGE (n)-[r:locatedAt{locZipCode: loc.nodeName, svcZipCode: n.zipCode}]->(loc) SET r += {
    createdDate: timestamp(),
    startTime: timestamp(),
    endTime: 4102444799000,
    status: "Confirmed"
}
RETURN COUNT(*)

MATCH (n:DREAM_KG{nodeType: 'Service'})
MATCH (loc:DREAM_KG{nodeType: 'Location'}) WHERE TOSTRING(n.zipCode) = loc.nodeName
MERGE (n)-[r:locatedAt{locZipCode: loc.nodeName, svcZipCode: n.zipCode}]->(loc) SET r += {
    createdDate: timestamp(),
    startTime: timestamp(),
    endTime: 4102444799000,
    status: "Confirmed"
}
RETURN COUNT(*)


## Location NodeType

In [6]:
with driver.session() as session:
  zip_codes = list(session.run('''MATCH (n:DREAM_KG{nodeType: "Service"})
  RETURN COLLECT(DISTINCT n.zipCode) AS zipCodes'''))[0]['zipCodes']

us_nomi = pgeocode.Nominatim('US')
ca_nomi = pgeocode.Nominatim('CA')

def get_city_for_postal(code):
    if isinstance(code, int):
        result = us_nomi.query_postal_code(code)
    elif isinstance(code, str):
        result = ca_nomi.query_postal_code(code)
    else:
        return None
    return result

locations = [get_city_for_postal(code) for code in zip_codes]
print(len(locations))

893


In [12]:
LOC_NODE_TYPE = 'Location'
with driver.session() as session:
  for location in locations:
    data = {
        "nodeName": location['postal_code'],
        "elementType": "instance",
        "nodeType": "Location",
        "nodeCategory": "DREAM_KG",
        "srcSheet": "pgeocode",
        "county": location['county_name'],
        "countryCode": location['country_code'],
        "countyCode": location['county_code'],
        "communityCode": location['community_code'],
        "community": location['community_name'],
        "zipCode": location['postal_code'],
        "accuracy": location['accuracy'],
        "state": location['state_name'],
        "stateCode": location['state_code'],
        "city": location['place_name'],
        "latitude": location['latitude'],
        "longitude": location['longitude']
    }
    loc_node = list(session.run(node_upsert(),
                              data=data))[0]['node']
    # root_cache_dict[LOC_NODE_TYPE][location['postal_code']] = {
    #     'identifier': loc_node.get('identifier'),
    #     'abbreviation': loc_node.get('abbreviation')
    # }

## Schedule NodeType

- Availability
  - Open
  - Closed

- Day of the Week

- Intervals
  - IntervalConstruct

- SlotHierarchy
  - SlotConstruct


In [33]:
# avail_set = set()
# day = set()
# intervals = set()
# for _, row in schedule_df[['schedule', 'schedule_json']].iterrows():
#   raw = row['schedule']
#   schedule = row['schedule_json']
#   if pd.isna(schedule):
#     print('*' * 121)
#   else:
#     schedule = schedule.replace('“', '"').replace(']], [[', '], [').replace('”', '"').strip(',')
#     data_list = json.loads(schedule)
#     for data in data_list:
#       avail_set.add(data[0].upper())
#       if len(data) >= 3:
#         day.add(data[1])
#         for i in range(2, len(data)):
#           intervals.add(data[i])
# print(avail_set)
# print(len(day))
# print(intervals)

## NLP NodeType

In [22]:
with driver.session() as session:
  session.run(node_upsert(subClass=True),
                          data={"nodeName": "NLPComputed",
                                "isRootNodeType": True,
                                "elementType": "class",
                                "nodeTypeName": "NLPComputed",
                                "nodeCategory": "DREAM_KG",
                                "metadataType": "nodeType",
                                "parentNodeType": "NLPComputed",
                                "childNodeType": "NLPComputed"})

In [14]:
with open('nodeDesc_rel_key.json', 'r') as fp:
  node_desc_rel_data = json.load(fp)

In [18]:
node_desc_rel_data['metaData']

{'name': 'NODEDESC_CONTAINS',
 'constraints': ['score'],
 'source': {'labels': ['DREAM_KG'],
  'constraints': ['nodeType', 'identifier', 'abbreviation']},
 'destination': {'labels': ['WordNGram'], 'constraints': ['title', 'version']}}

In [26]:
with driver.session() as session:
  for data in node_desc_rel_data['data']:
    newdata = {
        "nodeName": data['title'],
        "elementType": "instance",
        "nodeType": "NLPComputed",
        "nodeCategory": "DREAM_KG",
        "srcSheet": "nlpModule",
        "version": data['version'],
        "score": data['score']
    }
    try:
      nlp_node = list(session.run(node_upsert(),
                                data=newdata))[0]['node']

      nlp_meta = {
          'identifier': nlp_node.get('identifier'),
          'abbreviation': nlp_node.get('abbreviation')
      }
      session.run(business_rel_upsert(parent=nlp_meta,
                                      child={
                                          'identifier': data['identifier'],
                                          'abbreviation': data['abbreviation']
                                      },
                                      rel='nodeDescContains'), data={})
    except:
      continue

In [27]:
with open('notes_rel_key.json', 'r') as fp:
  notes_rel_data = json.load(fp)

with driver.session() as session:
  for data in notes_rel_data['data']:
    newdata = {
        "nodeName": data['title'],
        "elementType": "instance",
        "nodeType": "NLPComputed",
        "nodeCategory": "DREAM_KG",
        "srcSheet": "nlpModule",
        "version": data['version'],
        "score": data['score']
    }
    try:
      nlp_node = list(session.run(node_upsert(),
                                data=newdata))[0]['node']

      nlp_meta = {
          'identifier': nlp_node.get('identifier'),
          'abbreviation': nlp_node.get('abbreviation')
      }
      session.run(business_rel_upsert(parent=nlp_meta,
                                      child={
                                          'identifier': data['identifier'],
                                          'abbreviation': data['abbreviation']
                                      },
                                      rel='notesTextContains'), data={})
    except:
      continue

In [28]:
with open('requirements_rel_key.json', 'r') as fp:
  requirements_rel_data = json.load(fp)

with driver.session() as session:
  for data in requirements_rel_data['data']:
    newdata = {
        "nodeName": data['title'],
        "elementType": "instance",
        "nodeType": "NLPComputed",
        "nodeCategory": "DREAM_KG",
        "srcSheet": "nlpModule",
        "version": data['version'],
        "score": data['score']
    }
    try:
      nlp_node = list(session.run(node_upsert(),
                                data=newdata))[0]['node']

      nlp_meta = {
          'identifier': nlp_node.get('identifier'),
          'abbreviation': nlp_node.get('abbreviation')
      }
      session.run(business_rel_upsert(parent=nlp_meta,
                                      child={
                                          'identifier': data['identifier'],
                                          'abbreviation': data['abbreviation']
                                      },
                                      rel='requirementTextContains'), data={})
    except:
      continue

In [29]:
with open('recommendedFor_rel_key.json', 'r') as fp:
  recommendedFor_rel_data = json.load(fp)

with driver.session() as session:
  for data in recommendedFor_rel_data['data']:
    newdata = {
        "nodeName": data['title'],
        "elementType": "instance",
        "nodeType": "NLPComputed",
        "nodeCategory": "DREAM_KG",
        "srcSheet": "nlpModule",
        "version": data['version'],
        "score": data['score']
    }
    try:
      nlp_node = list(session.run(node_upsert(),
                                data=newdata))[0]['node']

      nlp_meta = {
          'identifier': nlp_node.get('identifier'),
          'abbreviation': nlp_node.get('abbreviation')
      }
      session.run(business_rel_upsert(parent=nlp_meta,
                                      child={
                                          'identifier': data['identifier'],
                                          'abbreviation': data['abbreviation']
                                      },
                                      rel='recommendedForTextContains'), data={})
    except:
      continue

In [30]:
with open('holidayException_rel_key.json', 'r') as fp:
  holidayException_rel_data = json.load(fp)

with driver.session() as session:
  for data in holidayException_rel_data['data']:
    newdata = {
        "nodeName": data['title'],
        "elementType": "instance",
        "nodeType": "NLPComputed",
        "nodeCategory": "DREAM_KG",
        "srcSheet": "nlpModule",
        "version": data['version'],
        "score": data['score']
    }
    try:
      nlp_node = list(session.run(node_upsert(),
                                data=newdata))[0]['node']

      nlp_meta = {
          'identifier': nlp_node.get('identifier'),
          'abbreviation': nlp_node.get('abbreviation')
      }
      session.run(business_rel_upsert(parent=nlp_meta,
                                      child={
                                          'identifier': data['identifier'],
                                          'abbreviation': data['abbreviation']
                                      },
                                      rel='holidayExceptionTextContains'), data={})
    except:
      continue